# 6장 데이터 로딩과 저장, 파일 형식

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
pd.options.display.max_colwidth = 75
pd.options.display.max_columns = 20
pd.options.display.max_rows = 10
np.set_printoptions(precision=4, suppress=True)
plt.rcParams["font.family"] = 'Malgun Gothic'

# 1. csv 텍스트 데이터
## 1.1 csv파일 읽기
* 판다스에서 CSV 파일을 읽기: pd.read_csv(path, encoding = 'utf-8') 함수를 사용
    * path 인수 : 파일시스템에서의 위치, URL, 파일객체를 나타내는 문자열
    * encoding 인수 - 유니코드 인코딩 종류을 지정


    1) header(컬럼명)가 있는 파일

In [2]:
#!cat examples/ex1.csv

df = pd.read_csv("examples/ex1.csv") # 데이터프레임으로 반환
df.head(2)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world


    2) header가 없는 파일 : 컬럼을 지정
        * names 인수 - 열 이름 리스트
        * header 인수 - 헤더가 없을 경우 None으로 지정

In [3]:
#!cat examples/ex2.csv
names = ["a", "b", "c", "d", "message"]
data = pd.read_csv("examples/ex2.csv", names=names)
data['message'] # Series

0    hello
1    world
2      foo
Name: message, dtype: object

    3) 계층적 색인 지정하기
        * index_col 인수 - 색인으로 사용할 열 번호나 이름, 계층적 색인을 지정할 경우 리스트

In [4]:
# !cat examples/csv_mindex.csv
parsed = pd.read_csv("examples/csv_mindex.csv",
                     index_col=["key1", "key2"]) #계층적 색인
parsed.columns

Index(['value1', 'value2'], dtype='object')

    4) 여러 개의 공백으로 필드를 구분한 파일 
        * sep 인수 - 필드을 구분하기 위해 사용할 정규 표현식

In [5]:
#!cat examples/ex3.txt

result = pd.read_csv("examples/ex3.txt",sep="\s+") #\s+ -> 스페이스바가 0개 이상
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


    5) 주석을 갖고 있는 파일
        * skiprows 인수 - 파일의 시작부터 무시할 행 수 또는 무시할 행 번호가 담긴 리스트

In [6]:
# !cat examples/ex4.csv
pd.read_csv("examples/ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


    6) **누락된 값이 있는 파일 : 비어 있는 문자열, NA, NULL을 NaN으로 출력** 
        * na_values 인수 - NA 값으로 처리할 값들의 목록

In [8]:
# !cat examples/ex5.csv

result = pd.read_csv("examples/ex5.csv")
result

#1) 누락된 값을 확인 : isna() 함수 이용 or 
# isnull(): 결측지를 True, 
# isnot(), notNa(): 결측치가 아닌 것을 True // 결측치 = null값을 의미함

pd.isna(result)

# 2) na_values -> 컬럼별 문자열 집합을 받아서 누락된 값 처리
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
result = pd.read_csv("examples/ex5.csv", na_values=sentinels)
print(result)
result.isna()

  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       NaN  5   6   NaN   8   world
2     three  9  10  11.0  12     NaN


,something,a,b,c,d,message
0,False,False,False,False,False,True
1,True,False,False,True,False,False
2,False,False,False,False,False,True


    7) 텍스트 파일 조금씩 
        * nrows 인수 - 파일의 첫 일부만 읽어올 때 처음 몇 줄을 읽을 것인지 지정 
        * chunksize 인수 - TextParser 객체에서 사용할 한 번에 읽을 파일의 크기

In [9]:
# result = pd.read_csv("examples/ex6.csv") # 판다스 출력 설정
# result
result = pd.read_csv("examples/ex6.csv", nrows=5) # 처음 몇 행만 읽기
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


### [예제]  ex6.csv 파일의 "key" 컬럼의 값들에 대한 빈도수을 계산하기

In [10]:
#!cat examples/ex6.csv
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)
# 파일을 읽은 후 chunksize 매개변수를 사용해 데이터를 1000개씩 나누어 처리함
chunker
tot = pd.Series([], dtype='int64')

# "key" 컬럼의 값들에 대한 누적의 합
for piece in chunker:# 10번 반복 수행
    # print(piece["key"].value_counts())
    tot = tot.add(piece["key"].value_counts(), fill_value=0)

tot
#값들을 정렬
tot = tot.sort_values(ascending=False)
tot
tot[:10]

key
E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

In [11]:
# 시리즈 간 덧셈 연산
from pandas import Series
tot = Series([], dtype= 'int64')
tot1 = Series([1,2,3])
#tot + tot1
# tot.add(tot1)
# 누적의 합
for i in range(3):
    tot = tot.add(tot1, fill_value = 0)
tot

0    3.0
1    6.0
2    9.0
dtype: float64

## 1.2. csv 텍스트 파일 저장하기 (매우 중요!!)
* 판다스에서 CSV 파일을 저장하기: df.to_csv(path, encoding = 'utf-8') 함수를 사용
    * path 인수 : 파일시스템에서의 위치, URL, 파일객체를 나타내는 문자열
    * encoding 인수 - 유니코드 인코딩 종류을 지정


In [13]:
# csv 파일로 저장
# 1. 데이터프레임 준비
data = pd.read_csv("examples/ex5.csv") # 원본에 대한 데이터프레임
data
data1 = data[['a','b','d']] # 컬럼 인덱싱 이용 데이터 추출
data1
# 2. 데이터프레임을 파일로 저장
# data1.to_csv("examples/out.csv")

,a,b,d
0,1,2,4
1,5,6,8
2,9,10,12


In [14]:
data.to_csv("examples/out.csv", sep="|") # 구분자 사용
data.to_csv("examples/out.csv", na_rep = "NULL") #누락된 값을 원하는 값으로 지정
data.to_csv("examples/out.csv", index=False, header=False)
data.to_csv("examples/out.csv", index=False, columns=["a", "b", "c"])


# 2. JSON  데이터
* JSON(JavaScript Object Notation)은 텍스트 데이터를 저장하고 전송하기 위한 일반적인 형식
* 주로 웹에서 데이터를 전송하거나 저장하는 데 사용
* JSON 파일은 일반적으로 텍스트 형식으로 작성되며, 사람과 기계 모두 이해하기 쉽다.
* JSON 파일은 키-값 쌍의 모음으로 구성
  * 키-값 쌍은 객체(object)라고도 하며, 중괄호 {} 로 묶는다.
  * 배열(array)은 대괄호 [] 안에 값의 목록을 나열하여 정의.

## 2.1 JSON 데이터 읽기/저장
    * pandas의 to_json() -> 데이터프레임을 JSON 파일로 저장
    * read_json() -> JSON 파일에서 데이터프레임을 읽어오기

### [예제]  pandas의 to_json() 및 read_json() 메서드를 사용하여 데이터프레임을 JSON 파일로 저장하고 읽기

In [15]:
import pandas as pd

# 1. JSON 파일로 저장할 데이터프레임 생성
data = {
    "name": ["John", "Alice", "Bob"],
    "age": [30, 25, 35],
    "city": ["New York", "Los Angeles", "Chicago"]
}
df = pd.DataFrame(data)

# 2. 데이터프레임을 JSON 파일로 저장
df.to_json("examples/data.json")

# 3. JSON 파일을 읽어와 데이터프레임으로 변환
df_read = pd.read_json("examples/data.json")

print("Original DataFrame:")
print(df)

print("\nDataFrame from JSON file:")
print(df_read)

Original DataFrame:
    name  age         city
0   John   30     New York
1  Alice   25  Los Angeles
2    Bob   35      Chicago

DataFrame from JSON file:
    name  age         city
0   John   30     New York
1  Alice   25  Los Angeles
2    Bob   35      Chicago


# 3. HTML 데이터
## 3.1 웹 스크래핑 - read_html()

* read_html() - html 문서 내 테이블 데이터 읽어 테이블 구조의 모든 데이터를 데이터프레임으로 저장해서 반환
    * https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/


### [예제] 미국 예금 보험 공사에서 부도 은행을 보여주는 HTML 문서의 테이블 데이터 읽어오기
    * 연도별로 부도 은행의 개수을 계산하시오. 

In [16]:
import pandas as pd
#tables = pd.read_html("examples/fdic_failed_bank_list.html") 

#1. 정적웹페이지 데이터 읽어오기
tables = pd.read_html("https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/")
# type = list

type(tables)
len(tables)  # 리스트에 포함된 테이블의 개수 ... 1개
failures = tables[0] # 인덱싱 - 첫 번째 테이블을 가져와 `failures`변수에 저장한 다음
failures
print(type(failures))
# 타입 = DataFrame  -  read_html() => `pandas` 라이브러리의 최상위 네임스페이스
# failures.head()

<class 'pandas.core.frame.DataFrame'>


In [17]:
# 2. 연도별로 부도 은행의 개수 : DataFrame 가공
# 1) 컬럼명 확인 /수정하기 /확인
failures.columns
failures.columns = ['BankNameBank', 'CityCity', 'StateSt', 'CertCert', 'AcquiringInstitutionAI', \
                    'ClosingDate', 'FundFund']
failures.head(3)

,BankNameBank,CityCity,StateSt,CertCert,AcquiringInstitutionAI,ClosingDate,FundFund
0,Citizens Bank,Sac City,IA,8758,Iowa Trust & Savings Bank,"November 3, 2023",10545
1,Heartland Tri-State Bank,Elkhart,KS,25851,"Dream First Bank, N.A.","July 28, 2023",10544
2,First Republic Bank,San Francisco,CA,59017,"JPMorgan Chase Bank, N.A.","May 1, 2023",10543


In [18]:
# 2) 데이터 변환 : 문자열 object -> datetime
failures["ClosingDate"] # 컬럼 인덱싱을 이용해서 시리즈로 추출
close_timestamps = pd.to_datetime(failures["ClosingDate"]) 
close_timestamps

# 3) 년도별 데이터 개수 추출
close_timestamps.dt.year
close_timestamps.dt.year.value_counts()

ClosingDate
2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2019      4
2001      4
2007      3
2003      3
2000      2
Name: count, Length: 19, dtype: int64

### [문제] : 네이버 주식 데이터을 스크래핑하여 csv파일로 저장하기 - read_html()
* https://www.naver.com/ -> 증권 -> 국내증시 -> 시가총액
    * https://finance.naver.com/sise/sise_market_sum.naver

In [ ]:
!pip install html5lib  lxml

In [19]:
import pandas as pd
# 1) 웹페이지 읽어오기 
# 'euc-kr' 또는 'cp949'로 인코딩 설정하여 페이지 읽기 시도
df_list = pd.read_html("https://finance.naver.com/sise/sise_market_sum.naver",encoding='cp949')
df_list
type(df_list)
df_list[1] # 인덱싱
# 데이터프레임 확인
for df in df_list:
    print("\n",df)


c:\users\user\appdata\local\programs\python\python39\lib\site-packages\bs4\__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")



      0      1          2        3          4       5
0  거래량   매수호가   거래대금(백만)  시가총액(억)    영업이익(억)  PER(배)
1   시가   매도호가      전일거래량  자산총계(억)    영업이익증가율  ROE(%)
2   고가  매수총잔량      외국인비율  부채총계(억)   당기순이익(억)  ROA(%)
3   저가  매도총잔량  상장주식수(천주)   매출액(억)   주당순이익(원)  PBR(배)
4  NaN    NaN        NaN   매출액증가율  보통주배당금(원)  유보율(%)

        N       종목명       현재가     전일비     등락률     액면가       시가총액      상장주식수  \
0    NaN       NaN       NaN     NaN     NaN     NaN        NaN        NaN   
1    1.0      삼성전자   84100.0   500.0  +0.60%   100.0  5020587.0  5969783.0   
2    2.0    SK하이닉스  188400.0  5500.0  +3.01%  5000.0  1371556.0   728002.0   
3    3.0  LG에너지솔루션  380000.0  6000.0  +1.60%   500.0   889200.0   234000.0   
4    4.0     삼성전자우   69200.0   900.0  +1.32%   100.0   569438.0   822887.0   
..   ...       ...       ...     ...     ...     ...        ...        ...   
76  49.0  HD한국조선해양  116300.0  2300.0  -1.94%  5000.0    82309.0    70773.0   
77  50.0      한화오션   26150.0  1100.0  -4.04%  5000.0  

In [20]:
# 2) 데이터프레임 가공 : csv파일로 저장하기
df = df_list[1]
type(df)

df.head(10)

# (1) 데이터프레임 탐색 :  종목명 컬럼명으로 인덱싱 - NaN 처리
df['종목명']

# (2) 조건색인
cond = df['종목명'].notnull() #불리언 시리즈
data = df[cond] # 불리언색인을 이용하여 필터링
data
data.head(10)

#3) 데이터프레임 저장
data.to_csv("examples/Naver_kospi.csv")

## 3.2 웹 스크래핑 - requests

* 웹 페이지 정보 가져오기 - requests 패키지 이용
  * requests.get()
  * requests.get().status_code
  * requests.get().text


### [예제] 패키지 requests을 활용하여 웹페이지 json 데이터을 데이터프레임으로 변환하기

In [ ]:
!pip install requests

In [ ]:
import requests
import pandas as pd
#1. 타겟 웹사이트 접속
url = "https://api.github.com/repos/pandas-dev/pandas/issues"
resp = requests.get(url)
resp.status_code
# 2. 웹페이지 콘텐츠 읽어오기
resp.text # 문자열로 반환
#pd.read_html(url)
# 3. json 내용을 딕셔너리로 변환
data = resp.json() # json의 내용을 딕셔너리 형태로 변환한 객체을 반환
print(data[0].keys()) # 키 정보 출력
# data[0]['number']

#4. 딕셔너리를 DataFrame으로 생성하기
issues = pd.DataFrame(data, columns=["number", "title",
                                      "labels", "state"])
issues.head()


### [문제] 네이버 블로그의 키워드 관련 검색 결과 텍스트 데이터 가져오기 - requests 모듈 사용시 한계점

In [ ]:
import requests
url = "https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=ALL&orderBy=sim&keyword=%ED%8C%8C%EC%9D%B4%EC%8D%AC"

# params = {
#     'pageNo' : 1,
#     'rangeType' : 'ALL',
#     'orderBy' : 'sim',
#     'keyword' : '파이썬'
# }

# response = requests.get('https://section.blog.naver.com/Search/Post.naver?', params=params)
response = requests.get(url)
print(response.status_code) # HTTP GET 방식으로 URL 파라미터 값 전달
print(response.url) # status_code 는 응답코드
print(response.text) # text에는 HTML 코드

## 3.3 웹 스크래핑 - beautifulsoup
* 웹 페이지 정보 추출하기 - beautifulsoup 사용법 
  * soup.select_one()
  * soup.select()
  * get_text()
  * get_attribute()
* 개발자 도구 사용

### [문제] 네이버 지식iN의 검색어 관련 검색 결과 제목 가져오기

In [4]:
import requests
from bs4 import BeautifulSoup

# 타겟 웹페이지 URL
url = 'https://kin.naver.com/search/list.naver?query=%ED%8C%8C%EC%9D%B4%EC%8D%AC'

# 웹페이지 로딩
response = requests.get(url)

if response.status_code == 200:
    html = response.text
    # html -> beautifulsoup 객체로 변환
    soup = BeautifulSoup(html, 'html.parser')

    # 제목 한개 선택하여 텍스트 가져오기
    title = soup.select_one('#s_content > div.section > ul > li:nth-child(1) > dl > dt > a')
    print(title.get_text())
    print(title.get_attribute_list("href"))

    print()

    # 제목 여러 개 선택하여 텍스트 리스트 가져오기
    titles = soup.select('#s_content > div.section > ul > li:nth-child(n) > dl > dt > a')
    for title in titles:
        print(title.get_text())
else : 
    print(response.status_code)



파이썬이 미래에 도움이 될까요?
['https://kin.naver.com/qna/detail.naver?d1id=3&dirId=31302&docId=466334742&qb=7YyM7J207I2s&enc=utf8§ion=kin&rank=1&search_sort=0&spq=0']

파이썬이 미래에 도움이 될까요?
중1 파이썬 독학
파이썬 사용
맥북 파이썬 idel
파이썬 질문
파이썬과 자바
파이썬 질문...
파이썬 배우기
파이썬강좌 독학할때 뭐보나요??
파이썬 id함수 질문이요


## 3.4 웹 스크래핑 - Selenium을 이용하여 크롤러( 6주차)
*  https://selenium-python.readthedocs.io/
* selenium 사용법
  1. 시스템의 OS 및 Chrome 브라우저의 버전 확인
  2. Chrome 드라이버 다운로드 및 설치
  3. 크롬 드라이버을 프로젝트 폴더에 저장 - 크롬 드라이버의 설치 경로
     * from selenium import webdriver
     * driver = webdriver.Chrome()
* 동적으로 웹 페이지 정보 추출하기
     * driver.get("웹주소") 
  

### [예제]Selenium을 이용하여 자동으로 타겟 웹 사이트 실행하기
* 타겟 사이트 : https://www.google.com

In [ ]:
!pip install selenium

In [ ]:
from selenium import webdriver
# 1. Chrome 브라우저 실행
driver = webdriver.Chrome() # 크롬 드라이버 실행파일의 위치 - 작업디렉터리
# 2. 타겟 웹페이지 로딩
url = "https://www.google.com"
driver.get(url)
# 3. 타겟 브라우저 종료
driver.quit()

### [문제] Selenium을 활용하여 타겟 웹사이트 자동으로 열기/ 쿼리로 검색하기/ 검색 결과의 제목과 url을 자동으로 추출하여 저장하는 web crawler 만들기
    * 사전 준비 작업 : 타겟 웹사이트을 구글 개발도구을 이용하여 탐색하기
    * 타겟 사이트: https://www.google.com
    * driver.find_element()
    * send_keys()

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time # Python 내장함수

# 1. Chrome 브라우저 실행
driver = webdriver.Chrome()

# 2. 타겟 Google 홈페이지 열기
url = "https://www.google.com"
driver.get(url)

# 3. 페이지가 완전히 로드될 때까지 2초 대기
time.sleep(2)

# 4. 검색 기능 - 검색창-> 검색어 입력 -> 엔터 키로 클릭 처리
try: # 네트워크 에러가 발생할 수 있기 때문에, 반드시 try, except, finally 구문으로 코딩하기
    
    # 4-1 검색 상자 요소 찾기
    search_box = driver.find_element(By.NAME, "q") 

    # 4-2 검색어 입력
    search_query = "OpenAI"
    search_box.send_keys(search_query) 

    # 4-3 Enter 키를 눌러 검색 수행
    search_box.send_keys(Keys.RETURN) # Selenium 엔터키 입력

    # 검색 결과가 로드될 때까지 5초 대기
    time.sleep(5)

    # 5. 검색에 관련된  결과 추출
    search_results = driver.find_elements(By.CSS_SELECTOR, "div.g") 
        
    # 5-1 첫 번째 제목과 링크 url 추출
    title = search_results[0].find_element(By.CSS_SELECTOR, "h3").text
    url = search_results[0].find_element(By.CSS_SELECTOR, "a").get_attribute("href")
    print("제목:", title)
    print("URL:", url)

    # 5-2 검색 결과의 모든 제목과 링크 URL 인쇄
    for result in search_results:
        title = result.find_element(By.CSS_SELECTOR, "h3").text
        url = result.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
        print("제목:", title)
        print("URL:", url)
    
except Exception as e:
    print("오류가 발생:", e)
    exit()

# finally:
#     # 브라우저 창 닫기
#     driver.quit()

제목: OpenAI
URL: https://openai.com/
제목: OpenAI
URL: https://openai.com/
제목: OpenAI
URL: https://openai.com/
제목: OpenAI
URL: https://namu.wiki/w/OpenAI
제목: OpenAI
URL: https://en.wikipedia.org/wiki/OpenAI
제목: 오픈AI - 위키백과, 우리 모두의 백과사전
URL: https://ko.wikipedia.org/wiki/%EC%98%A4%ED%94%88AI
제목: OpenAI
URL: https://twitter.com/OpenAI
제목: OpenAI
URL: https://www.linkedin.com/company/openai
제목: 설명
URL: https://www.google.com/search?q=OpenAI&sca_esv=835ba0ee2498b9ce&source=hp&ei=SpkXZr3KEtWNvr0P7t-roAU&iflsig=ANes7DEAAAAAZhenWgrDXFtWs_vVaXOU9AjPSF2oUIFM&udm=&ved=0ahUKEwi9l5i22bmFAxXVhq8BHe7vClQQ4dUDCA8&uact=5&oq=OpenAI&gs_lp=Egdnd3Mtd2l6IgZPcGVuQUlIWlAAWB5wAHgAkAEAmAEAoAEAqgEAuAEDyAEA-AEBmAIAoAIAmAMAkgcAoAcA&sclient=gws-wiz#


### [문제] Selenium을 이용하여 네이버 증권 웹페이지 열기/ 삼성전자 일별 시세 HTML가져오기/ 10페이지 내역 테이블 데이터 찾기/검색 결과를 데이터프레임으로 변환/저장하기
* 타겟 사이트: https://finance.naver.com/item/sise.naver?code=005930* 

In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

In [6]:
#1. 크롬 드라이버 실행
driver= webdriver.Chrome()

# 2페이지 로딩 시간
time.sleep(2)

# 3. 크롤링 사이트 : 삼성전자의 일별 시세 페이지 URL 찾기
samsung_url ="https://finance.naver.com/item/sise_day.naver?code=005930&page=" #base url

# 4. 첫 페이지부터 10 페이지까지 테이블 담기
samsung_daily_kospi = [] # 리스트 선언
for num in range(1,11):
    full_url = samsung_url + str(num)
    # 4.1 해당 페이지 요청
    driver.get(full_url)
        
    try:
        # 4.2 페이지의 HTML 가져오기
        html = driver.page_source
        df = pd.read_html(html) # 테이블 구조 -> 데이터프레임
        # print(df) #df는 리스트 형
        
        # 4.3 결측치 처리 - 조건 색인을 이용한 불리언 인덱싱
        ## 조건 색인
        cond = df[0]['날짜'].notnull() #df의 0번째 인덱 
        samsung_day = df[0][cond]
        ## 리스트에 담기
        samsung_daily_kospi.append(samsung_day)
                
    except Exception as e:
        print("Network Error",e)
    
    # finally: 
    #     driver.quit()

 # 5. 데이터프레임 합치기
samsung_daily_kospi_all = pd.concat(samsung_daily_kospi, ignore_index=True)
samsung_daily_kospi_all.head(20)
# 6. csv 파일로 저장
# samsung_daily_kospi_all.to_csv("samsung_daily_kospi_2024_04_11.csv")

C:\Users\user\AppData\Local\Temp\ipykernel_2424\3595332979.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html) # 테이블 구조 -> 데이터프레임
C:\Users\user\AppData\Local\Temp\ipykernel_2424\3595332979.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html) # 테이블 구조 -> 데이터프레임
C:\Users\user\AppData\Local\Temp\ipykernel_2424\3595332979.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html) # 테이블 구조 -> 데이터프레임
C:\Users\user\AppData\Local\Temp\ipykernel_2424\3595332979.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a futu

,날짜,종가,전일비,시가,고가,저가,거래량
0,2024.04.11,84100.0,500.0,83200.0,84700.0,82500.0,25337989.0
1,2024.04.09,83600.0,900.0,84500.0,84900.0,83100.0,23725956.0
2,2024.04.08,84500.0,0.0,85200.0,86000.0,84500.0,18953232.0
3,2024.04.05,84500.0,800.0,84500.0,85000.0,83800.0,18883752.0
4,2024.04.04,85300.0,1200.0,85200.0,85500.0,84300.0,25248934.0
5,2024.04.03,84100.0,900.0,84300.0,85000.0,83500.0,30493347.0
6,2024.04.02,85000.0,3000.0,82900.0,85000.0,82900.0,37077944.0
7,2024.04.01,82000.0,400.0,83200.0,83300.0,82000.0,20116513.0
8,2024.03.29,82400.0,1600.0,81200.0,82500.0,80900.0,27126366.0
9,2024.03.28,80800.0,1000.0,79400.0,81000.0,79200.0,25084812.0
